In [5]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd
import skforecast

print(skforecast.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/joaquin/Documents/GitHub/skforecast
0.19.0


In [7]:
import numpy as np
import pandas as pd
import joblib
import nannyml as nml
from sklearn.ensemble import RandomForestRegressor
from skforecast.datasets import fetch_dataset
from skforecast.drift_detection import PopulationDriftDetector

In [10]:
data = joblib.load(r'/home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/tests/tests_population_drift/fixture_data_population_drift.joblib')
data_train = data.iloc[: len(data) // 2].copy()
data_new  = data.iloc[len(data) // 2 :].copy()
data_train['weather'] = data_train['weather'].astype('category')
data_new['weather'] = pd.Categorical(data_new['weather'], categories=data_train['weather'].cat.categories)

In [43]:
detector = nml.UnivariateDriftCalculator(
    column_names=data_train.columns.tolist(),
    timestamp_column_name="date_time",
    chunk_period='M',
    categorical_methods=['chi2', 'jensen_shannon'],
    continuous_methods=['kolmogorov_smirnov', 'jensen_shannon'],
)
detector.fit(reference_data=data_train.reset_index())
results_nannyml = detector.calculate(data=data_new.reset_index())
results_nannyml = results_nannyml.filter(period='reference').to_df(multilevel=False)
results_nannyml

,chunk_key,chunk_index,chunk_start_index,chunk_end_index,chunk_start_date,chunk_end_date,chunk_period,atemp_kolmogorov_smirnov_value,atemp_kolmogorov_smirnov_upper_threshold,atemp_kolmogorov_smirnov_lower_threshold,...,workingday_jensen_shannon_lower_threshold,workingday_jensen_shannon_alert,weather_chi2_value,weather_chi2_upper_threshold,weather_chi2_lower_threshold,weather_chi2_alert,weather_jensen_shannon_value,weather_jensen_shannon_upper_threshold,weather_jensen_shannon_lower_threshold,weather_jensen_shannon_alert
0,2011-01,0,0,743,2011-01-01,2011-01-31 23:59:59.999999999,reference,0.700782,0.95185,None,...,None,False,17.343127,None,None,True,0.066402,0.447542,None,False
1,2011-02,1,744,1415,2011-02-01,2011-02-28 23:59:59.999999999,reference,0.477782,0.95185,None,...,None,False,1.590640,None,None,False,0.020856,0.447542,None,False
2,2011-03,2,1416,2159,2011-03-01,2011-03-31 23:59:59.999999999,reference,0.448033,0.95185,None,...,None,False,26.319459,None,None,True,0.081522,0.447542,None,False
3,2011-04,3,2160,2879,2011-04-01,2011-04-30 23:59:59.999999999,reference,0.158069,0.95185,None,...,None,False,21.560289,None,None,True,0.075032,0.447542,None,False
4,2011-05,4,2880,3623,2011-05-01,2011-05-31 23:59:59.999999999,reference,0.346800,0.95185,None,...,None,False,0.685494,None,None,False,0.013682,0.447542,None,False
5,2011-06,5,3624,4343,2011-06-01,2011-06-30 23:59:59.999999999,reference,0.561375,0.95185,None,...,None,False,30.129183,None,None,True,0.098200,0.447542,None,False
6,2011-07,6,4344,5087,2011-07-01,2011-07-31 23:59:59.999999999,reference,0.660198,0.95185,None,...,None,False,136.738895,None,None,True,0.216060,0.447542,None,False
7,2011-08,7,5088,5831,2011-08-01,2011-08-31 23:59:59.999999999,reference,0.604329,0.95185,None,...,None,False,38.850946,None,None,True,0.106581,0.447542,None,False
8,2011-09,8,5832,6551,2011-09-01,2011-09-30 23:59:59.999999999,reference,0.451334,0.95185,None,...,None,False,85.426361,None,None,True,0.147994,0.447542,None,False
9,2011-10,9,6552,7295,2011-10-01,2011-10-31 23:59:59.999999999,reference,0.238413,0.95185,None,...,None,False,12.528396,None,None,True,0.054190,0.447542,None,False


In [42]:
results_nannyml.filter(regex='chunk|weather')

,chunk_key,chunk_index,chunk_start_index,chunk_end_index,chunk_start_date,chunk_end_date,chunk_period,weather_chi2_value,weather_chi2_upper_threshold,weather_chi2_lower_threshold,weather_chi2_alert,weather_jensen_shannon_value,weather_jensen_shannon_upper_threshold,weather_jensen_shannon_lower_threshold,weather_jensen_shannon_alert
0,2011-01,0,0,743,2011-01-01,2011-01-31 23:59:59.999999999,reference,17.343127,None,None,True,0.066402,0.447542,None,False
1,2011-02,1,744,1415,2011-02-01,2011-02-28 23:59:59.999999999,reference,1.590640,None,None,False,0.020856,0.447542,None,False
2,2011-03,2,1416,2159,2011-03-01,2011-03-31 23:59:59.999999999,reference,26.319459,None,None,True,0.081522,0.447542,None,False
3,2011-04,3,2160,2879,2011-04-01,2011-04-30 23:59:59.999999999,reference,21.560289,None,None,True,0.075032,0.447542,None,False
4,2011-05,4,2880,3623,2011-05-01,2011-05-31 23:59:59.999999999,reference,0.685494,None,None,False,0.013682,0.447542,None,False
5,2011-06,5,3624,4343,2011-06-01,2011-06-30 23:59:59.999999999,reference,30.129183,None,None,True,0.098200,0.447542,None,False
6,2011-07,6,4344,5087,2011-07-01,2011-07-31 23:59:59.999999999,reference,136.738895,None,None,True,0.216060,0.447542,None,False
7,2011-08,7,5088,5831,2011-08-01,2011-08-31 23:59:59.999999999,reference,38.850946,None,None,True,0.106581,0.447542,None,False
8,2011-09,8,5832,6551,2011-09-01,2011-09-30 23:59:59.999999999,reference,85.426361,None,None,True,0.147994,0.447542,None,False
9,2011-10,9,6552,7295,2011-10-01,2011-10-31 23:59:59.999999999,reference,12.528396,None,None,True,0.054190,0.447542,None,False


In [47]:
detector = PopulationDriftDetector(
        chunk_size='MS',            
        threshold=3,
        threshold_method='std'
    )
detector.fit(data_train)
results_skforecast, _ = detector.predict(data_new)

In [58]:
np.testing.assert_array_equal(

results_nannyml['weather_chi2_value'].to_numpy(),
detector.empirical_dist_chi2_['weather']
)

AssertionError: 
Arrays are not equal

Mismatched elements: 2 / 13 (15.4%)
Max absolute difference among violations: 3.55271368e-15
Max relative difference among violations: 1.61959622e-16
 ACTUAL: array([ 17.343127,   1.59064 ,  26.319459,  21.560289,   0.685494,
        30.129183, 136.738895,  38.850946,  85.426361,  12.528396,
         0.738841,   2.193382,   6.491772])
 DESIRED: array([ 17.343127,   1.59064 ,  26.319459,  21.560289,   0.685494,
        30.129183, 136.738895,  38.850946,  85.426361,  12.528396,
         0.738841,   2.193382,   6.491772])

In [ ]:
results_nannyml['weather_jensen_shannon_value'].to_numpy() == detector.empirical_dist_js_['weather']

array([0.06640221, 0.02085606, 0.08152191, 0.0750316 , 0.01368161,
       0.09820035, 0.21605988, 0.10658109, 0.1479944 , 0.05418984,
       0.01417818, 0.02476808, 0.45034392])

In [50]:
detector.empirical_dist_js_['weather']

[np.float64(0.055283465821809305),
 np.float64(0.017363809343793148),
 np.float64(0.06787143920384266),
 np.float64(0.06246790112483091),
 np.float64(0.01139068844479754),
 np.float64(0.08175715559564375),
 np.float64(0.17988165210528834),
 np.float64(0.08873457799603018),
 np.float64(0.12321342096210208),
 np.float64(0.04511600157986298),
 np.float64(0.011804105479578768),
 np.float64(0.02062078251073618),
 np.float64(0.3749359099849188)]

In [41]:
results_skforecast.query("feature == 'weather'")

,chunk,chunk_start,chunk_end,feature,ks_statistic,ks_threshold,chi2_statistic,chi2_threshold,js_statistic,js_threshold,reference_range,is_out_of_range,drift_ks_statistic,drift_chi2_statistic,drift_js,drift_detected
24,0,2012-01-01 12:00:00,2012-01-31 23:00:00,weather,NaN,NaN,2.699410,143.772203,0.026844,0.372603,"(nan, nan)",NaN,False,False,False,False
25,1,2012-02-01 00:00:00,2012-02-29 23:00:00,weather,NaN,NaN,0.319118,143.772203,0.009387,0.372603,"(nan, nan)",NaN,False,False,False,False
26,2,2012-03-01 00:00:00,2012-03-31 23:00:00,weather,NaN,NaN,9.970343,143.772203,0.055408,0.372603,"(nan, nan)",NaN,False,False,False,False
27,3,2012-04-01 00:00:00,2012-04-30 23:00:00,weather,NaN,NaN,15.945695,143.772203,0.067434,0.372603,"(nan, nan)",NaN,False,False,False,False
28,4,2012-05-01 00:00:00,2012-05-31 23:00:00,weather,NaN,NaN,3.885379,143.772203,0.031668,0.372603,"(nan, nan)",NaN,False,False,False,False
29,5,2012-06-01 00:00:00,2012-06-30 23:00:00,weather,NaN,NaN,50.988327,143.772203,0.124303,0.372603,"(nan, nan)",NaN,False,False,False,False
30,6,2012-07-01 00:00:00,2012-07-31 23:00:00,weather,NaN,NaN,17.060054,143.772203,0.069940,0.372603,"(nan, nan)",NaN,False,False,False,False
31,7,2012-08-01 00:00:00,2012-08-31 23:00:00,weather,NaN,NaN,17.541022,143.772203,0.074891,0.372603,"(nan, nan)",NaN,False,False,False,False
32,8,2012-09-01 00:00:00,2012-09-30 23:00:00,weather,NaN,NaN,28.580204,143.772203,0.091716,0.372603,"(nan, nan)",NaN,False,False,False,False
33,9,2012-10-01 00:00:00,2012-10-31 23:00:00,weather,NaN,NaN,35.409177,143.772203,0.094290,0.372603,"(nan, nan)",NaN,False,False,False,False


In [9]:
results_skforecast['dummy'] = np.nan
results_skforecast.dtypes

chunk                            int64
chunk_start             datetime64[ns]
chunk_end               datetime64[ns]
feature                         object
ks_statistic                   float64
ks_threshold                   float64
chi2_statistic                 float64
chi2_threshold                 float64
js_statistic                   float64
js_threshold                   float64
reference_range                 object
is_out_of_range                 object
drift_ks_statistic                bool
drift_chi2_statistic              bool
drift_js                          bool
drift_detected                    bool
dummy                          float64
dtype: object

In [44]:
from scipy import stats

# Tamaño de la muestra de referencia (datos de entrenamiento)
n1 = len(data_train)

# Tamaño de un chunk (un mes de datos nuevos)
# NannyML usa chunk_period='M', veamos cuántos registros tiene cada mes
chunk_sizes = data_new.groupby(pd.Grouper(freq='M')).size()
print("Tamaño de cada chunk (por mes):")
print(chunk_sizes)
print()

# Nivel de significancia por defecto en NannyML
alpha = 0.05

# Valor crítico de la distribución KS
c_alpha = stats.kstwobign.ppf(1 - alpha)
print(f"Valor crítico c(alpha=0.05): {c_alpha:.4f}")
print()

# Calcular threshold para cada chunk
print("Threshold KS para cada chunk:")
for chunk_name, n2 in chunk_sizes.items():
    threshold = c_alpha * np.sqrt((n1 + n2) / (n1 * n2))
    print(f"  {chunk_name.strftime('%Y-%m')}: n2={n2}, threshold={threshold:.6f}")

print()
print(f"n1 (referencia) = {n1}")
print()

# Comparar con el threshold de NannyML para 'atemp'
print("Threshold de NannyML para 'atemp_kolmogorov_smirnov':")
print(results_nannyml[['chunk_start_date', 'atemp_kolmogorov_smirnov_upper_threshold']].to_string())

Tamaño de cada chunk (por mes):
date_time
2012-01-31    732
2012-02-29    696
2012-03-31    744
2012-04-30    720
2012-05-31    744
2012-06-30    720
2012-07-31    744
2012-08-31    744
2012-09-30    720
2012-10-31    744
2012-11-30    720
2012-12-31    744
Freq: ME, dtype: int64

Valor crítico c(alpha=0.05): 1.3581

Threshold KS para cada chunk:
  2012-01: n2=732, threshold=0.052249
  2012-02: n2=696, threshold=0.053482
  2012-03: n2=744, threshold=0.051859
  2012-04: n2=720, threshold=0.052650
  2012-05: n2=744, threshold=0.051859
  2012-06: n2=720, threshold=0.052650
  2012-07: n2=744, threshold=0.051859
  2012-08: n2=744, threshold=0.051859
  2012-09: n2=720, threshold=0.052650
  2012-10: n2=744, threshold=0.051859
  2012-11: n2=720, threshold=0.052650
  2012-12: n2=744, threshold=0.051859

n1 (referencia) = 8772

Threshold de NannyML para 'atemp_kolmogorov_smirnov':
   chunk_start_date  atemp_kolmogorov_smirnov_upper_threshold
0        2012-01-01                                   

C:\Users\jaesc2\AppData\Local\Temp\ipykernel_32776\405903868.py:8: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  chunk_sizes = data_new.groupby(pd.Grouper(freq='M')).size()


All 1 retries exhausted. Final error: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get issuer certificate (_ssl.c:1010)')))
error uploading: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get issuer certificate (_ssl.c:1010)')))


In [10]:
# NannyML usa un enfoque diferente: calcula el threshold basándose en los 
# valores del estadístico KS calculados en los DATOS DE REFERENCIA (no en los de análisis)
# Usa percentiles de la distribución empírica del estadístico KS en referencia

# Veamos qué valores de KS obtiene en los chunks de referencia
detector_ref = nml.UnivariateDriftCalculator(
    column_names=['atemp'],
    timestamp_column_name="date_time",
    chunk_period='M',
    continuous_methods=['kolmogorov_smirnov'],
)
detector_ref.fit(reference_data=data_train.reset_index())
results_ref = detector_ref.calculate(data=data_train.reset_index())
results_ref_df = results_ref.filter(period='reference').to_df(multilevel=False)

print("Valores del estadístico KS en los chunks de REFERENCIA:")
print(results_ref_df[['chunk_start_date', 'atemp_kolmogorov_smirnov_value']].to_string())
print()

# NannyML calcula el threshold usando percentiles de estos valores
# Por defecto usa el percentil 95 como upper threshold
ks_values_ref = results_ref_df['atemp_kolmogorov_smirnov_value'].values
threshold_percentile_95 = np.percentile(ks_values_ref, 95)
print(f"Percentil 95 de los valores KS en referencia: {threshold_percentile_95:.5f}")
print(f"Threshold de NannyML: {results_nannyml['atemp_kolmogorov_smirnov_upper_threshold'].iloc[0]:.5f}")

Valores del estadístico KS en los chunks de REFERENCIA:
   chunk_start_date  atemp_kolmogorov_smirnov_value
0        2011-01-01                        0.700782
1        2011-02-01                        0.477782
2        2011-03-01                        0.448033
3        2011-04-01                        0.158069
4        2011-05-01                        0.346800
5        2011-06-01                        0.561375
6        2011-07-01                        0.660198
7        2011-08-01                        0.604329
8        2011-09-01                        0.451334
9        2011-10-01                        0.238413
10       2011-11-01                        0.359118
11       2011-12-01                        0.469829
12       2012-01-01                        0.662905

Percentil 95 de los valores KS en referencia: 0.67806
Threshold de NannyML: 0.95185


All 1 retries exhausted. Final error: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get issuer certificate (_ssl.c:1010)')))
error uploading: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get issuer certificate (_ssl.c:1010)')))
All 1 retries exhausted. Final error: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get issuer certificate (_ssl.c:1010)')))
error uploading: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by SSLError(SSLCertVer

In [13]:
# NannyML usa el método "standard deviation" por defecto para calcular thresholds
# La fórmula es: threshold = mean ± (std_threshold * std)
# donde std_threshold por defecto es 3 (3 desviaciones estándar)

# Calculemos manualmente usando los valores de referencia
ks_values_ref = results_ref_df['atemp_kolmogorov_smirnov_value'].values
mean_ks = np.mean(ks_values_ref)

# Probemos con ddof=0 (population std, que es lo que usa numpy por defecto)
std_ks_ddof0 = np.std(ks_values_ref, ddof=0)
std_ks_ddof1 = np.std(ks_values_ref, ddof=1)

# NannyML usa 3 desviaciones estándar por defecto
std_threshold = 3

print(f"Media de KS en referencia: {mean_ks:.5f}")
print(f"Std (ddof=0): {std_ks_ddof0:.5f}")
print(f"Std (ddof=1): {std_ks_ddof1:.5f}")
print()

upper_ddof0 = mean_ks + std_threshold * std_ks_ddof0
upper_ddof1 = mean_ks + std_threshold * std_ks_ddof1

print(f"Upper threshold (ddof=0): {upper_ddof0:.5f}")
print(f"Upper threshold (ddof=1): {upper_ddof1:.5f}")
print(f"Upper threshold NannyML:  {results_nannyml['atemp_kolmogorov_smirnov_upper_threshold'].iloc[0]:.5f}")

Media de KS en referencia: 0.47223
Std (ddof=0): 0.15987
Std (ddof=1): 0.16640

Upper threshold (ddof=0): 0.95185
Upper threshold (ddof=1): 0.97143
Upper threshold NannyML:  0.95185


In [16]:
# =============================================================================
# Cálculo del threshold usando StandardDeviationThreshold de NannyML
# =============================================================================
# NannyML calcula los thresholds de la siguiente manera:
#
# 1. Divide los datos de REFERENCIA en chunks (en este caso, por mes)
# 2. Calcula el estadístico (ej: KS) para cada chunk de referencia
# 3. Usa StandardDeviationThreshold para calcular los límites:
#
#    upper_threshold = np.nanmean(data) + std_upper_multiplier * np.nanstd(data)
#    lower_threshold = np.nanmean(data) - std_lower_multiplier * np.nanstd(data)
#
# Por defecto: std_lower_multiplier = std_upper_multiplier = 3

from nannyml.thresholds import StandardDeviationThreshold

# Crear instancia del threshold (valores por defecto)
std_threshold = StandardDeviationThreshold(
    std_lower_multiplier=3,
    std_upper_multiplier=3,
    offset_from=np.nanmean  # Por defecto usa la media
)

# Calcular thresholds usando los valores KS de los chunks de referencia
lower_calc, upper_calc = std_threshold.thresholds(data=ks_values_ref)

print("=== Cálculo usando StandardDeviationThreshold de NannyML ===")
print(f"Datos de entrada (KS de chunks de referencia): {ks_values_ref.round(4)}")
print()
print(f"np.nanmean(data) = {np.nanmean(ks_values_ref):.5f}")
print(f"np.nanstd(data)  = {np.nanstd(ks_values_ref):.5f}")
print()
print(f"Lower threshold calculado: {lower_calc:.5f}")
print(f"Upper threshold calculado: {upper_calc:.5f}")
print()
print(f"Upper threshold de NannyML (atemp_kolmogorov_smirnov): {results_nannyml['atemp_kolmogorov_smirnov_upper_threshold'].iloc[0]:.5f}")
calc_threshold = np.nanmean(ks_values_ref) + 3 * np.nanstd(ks_values_ref)
print(f"Theoretical upper threshold: {calc_threshold}")
print()
print("¿Coinciden?", np.isclose(upper_calc, calc_threshold))

=== Cálculo usando StandardDeviationThreshold de NannyML ===
Datos de entrada (KS de chunks de referencia): [0.7008 0.4778 0.448  0.1581 0.3468 0.5614 0.6602 0.6043 0.4513 0.2384
 0.3591 0.4698 0.6629]

np.nanmean(data) = 0.47223
np.nanstd(data)  = 0.15987

Lower threshold calculado: -0.00739
Upper threshold calculado: 0.95185

Upper threshold de NannyML (atemp_kolmogorov_smirnov): 0.95185
Theorical upper threslhold: 0.9518496626499939

¿Coinciden? True


In [18]:
# Sample DataFrame
df = pd.DataFrame({
    'A': np.random.rand(100),
    'B': np.random.rand(100)
})
df.head(2)

,A,B
0,0.952656,0.369023
1,0.416993,0.371978


In [ ]:
df.loc[pd.Index(range(10)), 'B'].dropna()

0    0.369023
1    0.371978
2    0.581042
3    0.181109
4    0.140600
5    0.094024
6    0.281634
7    0.330321
8    0.790694
9    0.674570
Name: B, dtype: float64

In [20]:
df.loc[pd.Index([1, 10000]), 'B'].dropna()

KeyError: '[10000] not in index'

In [21]:
np.mean([1, 2 ,3 ])

np.float64(2.0)

In [22]:
index = pd.date_range('1/1/2000', periods=9, freq='min')
series = pd.Series(range(9), index=index)
series

2000-01-01 00:00:00    0
2000-01-01 00:01:00    1
2000-01-01 00:02:00    2
2000-01-01 00:03:00    3
2000-01-01 00:04:00    4
2000-01-01 00:05:00    5
2000-01-01 00:06:00    6
2000-01-01 00:07:00    7
2000-01-01 00:08:00    8
Freq: min, dtype: int64

In [23]:
series.resample('3min').sum()

2000-01-01 00:00:00     3
2000-01-01 00:03:00    12
2000-01-01 00:06:00    21
Freq: 3min, dtype: int64

In [24]:
series.resample(pd.DateOffset(minutes=3)).sum()

NotImplementedError: Prefix not defined

In [25]:
pd.DateOffset(minutes=3)

<DateOffset: minutes=3>

In [33]:
detector = PopulationDriftDetector(
               chunk_size = '3min',  # Monthly chunks
               threshold  = 0.95
           )

In [36]:
detector.chunk_size

<3 * Minutes>

In [37]:
pd.DateOffset(minutes=3)

<DateOffset: minutes=3>